# VacationPy
![Hotel Map](/Images/hotel_map.png)
----

# Observations: 
* Loaded citiy data from the cities data CSV

* Created a data file of ideeal cities:
Tempreature  >= 70 degrees but <= 85 degrees
Humidity >= 70%
cloudinness <= 10

* Made call to Google Places API to return the nearest Hotel Names within 5000 meters of the lat lng for the city
* Created a heat map of the Huimidity
* Created markers to pmark the locations of the hotel names

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Part II - VacationPy

Now let's use your skills in working with weather data to plan future vacations. Use jupyter-gmaps and the Google Places API for this part of the assignment.

* **Note:** Remember that any API usage beyond the $200 credit will be charged to your personal account. You can set quotas and limits to your daily requests to be sure you can't be charged. Check out [Google Maps Platform Billing](https://developers.google.com/maps/billing/gmp-billing#monitor-and-restrict-consumption) and [Manage your cost of use](https://developers.google.com/maps/documentation/javascript/usage-and-billing#set-caps) for more information.

* **Note:** if you having trouble displaying the maps, try running `jupyter nbextension enable --py gmaps` in your environment and retry.

To complete this part of the assignment,you will need to do the following:

* Create a heat map that displays the humidity for every city from Part I.

  ![heatmap](Images/heatmap.png)

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

  * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

* Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

* Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

  ![hotel map](Images/hotel_map.png)

As final considerations:

* You must complete your analysis using a Jupyter notebook.
* You must use the Matplotlib or Pandas plotting libraries.
* For Part I, you must include a written description of three observable trends based on the data.
* For Part II, you must include a screenshot of the heatmap you create and include it in your submission.
* You must use proper labeling of your plots, including aspects like: Plot Titles (with date of analysis) and Axes Labels.
* For max intensity in the heat map, try setting it to the highest humidity found in the data set.

## Hints and Considerations

* The city data you generate is based on random coordinates as well as different query times. As such, your outputs will not be an exact match to the provided starter notebook.

* If you'd like a refresher on the geographic coordinate system, [this site](http://desktop.arcgis.com/en/arcmap/10.3/guide-books/map-projections/about-geographic-coordinate-systems.htm) has great information.

* Next, spend the requisite time necessary to study the OpenWeatherMap API. Based on your initial study, you should be able to answer basic questions about the API: Where do you request the API key? Which Weather API in particular will you need? What URL endpoints does it expect? What JSON structure does it respond with? Before you write a line of code, you should be aiming to have a crystal clear understanding of your intended outcome.

* A starter code for Citipy has been provided. However, if you're craving an extra challenge, push yourself to learn how it works: [citipy Python library](https://pypi.python.org/pypi/citipy). Before you try to incorporate the library into your analysis, start by creating simple test cases outside your main script to confirm that you are using it correctly. Too often, when introduced to a new library, students get bogged down by the most minor of errors -- spending hours investigating their entire code -- when, in fact, a simple and focused test would have shown their basic utilization of the library was wrong from the start. Don't let this be you!

* Part of our expectation in this challenge is that you will use critical thinking skills to understand how and why we're recommending the tools we are. What is Citipy for? Why would you use it in conjunction with the OpenWeatherMap API? How would you do so?

* In building your script, pay attention to the cities you are using in your query pool. Are you getting coverage of the full gamut of latitudes and longitudes? Or are you simply choosing 500 cities concentrated in one region of the world? Even if you were a geographic genius, simply rattling 500 cities based on your human selection would create a biased dataset. Be thinking of how you should counter this. (Hint: Consider the full range of latitudes).

* Once you have computed the linear regression for one chart, the process will be similar for all others. As a bonus, try to create a function that will create these charts based on different parameters.

* Remember that each coordinate will trigger a separate call to the Google API. If you're creating your own criteria to plan your vacation, try to reduce the results in your DataFrame to 10 or fewer cities.

* Ensure your repository has regular commits (i.e. 20+ commits) and a thorough README.md file.

* Lastly, remember -- this is a challenging activity. Push yourself! If you complete this task, then you can safely say that you've gained a strong mastery of the core foundations of data analytics and it will only go better from here. Good luck!

### Copyright

Trilogy Education Services Â© 2020. All Rights Reserved.







In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json
from citipy import citipy

# Import API key
from api_keys import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the output_data/cities CSV into a data frame
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#Create a list using Lat Lng columns as coordinates
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

In [14]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'hieght': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2, center=(15,25))
# Create the heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=1.5)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
plt.savefig('../images.Humidity_Heatmap.png')
fig


#fig(layout=figurelayout(height='420px')

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Narrow weather criteria to Max Temp < 85 and > 70 degrees
# Humidity > 70% 
# Wind Speed < 10 mph
# Cloudiness of 10 or less
# Drop any cities that don't meet all four criteria

fitting_weather = cities_df.dropna()
fitted_cities = fitting_weather.loc[(fitting_weather['Humidity'] >= 70) & (fitting_weather['Cloudiness'] <= 10)]


fitted_cities


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
11,11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
13,13,saskylakh,0,RU,1558378757,91,71.97,114.09,4.92,2.26
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
40,40,hermanus,9,ZA,1558378761,74,-34.42,19.24,55.00,23.78
55,55,khatanga,10,RU,1558378764,92,71.98,102.47,9.96,10.94
57,57,kolyvan,0,RU,1558378764,87,55.31,82.74,48.20,4.47
84,84,emerald,0,AU,1558378770,82,-23.53,148.16,60.80,6.93
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52


In [16]:

last_fitted_cities = fitted_cities.loc[(fitted_cities['Max Temp'] >= 70) & (fitted_cities['Max Temp'] <= 85)]


last_fitted_cities

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
118,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
326,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50
337,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41
391,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
402,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df = last_fitted_cities

hotel_df
                        

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
20,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
118,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
326,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50
337,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41
391,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
402,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50


In [18]:
# Add Hotel Name as an additional column
hotel_df['Hotel Name'] 
hotel_df.head()

KeyError: 'Hotel Name'

In [19]:
# Reset Index
indexed_hotel_df = hotel_df.reset_index()
del indexed_hotel_df['index']
indexed_hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23
1,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
2,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91
3,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
5,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
6,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50
7,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40
9,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50


In [20]:
hotel = []

for city in range(len(indexed_hotel_df["City"])):
    

    lat = indexed_hotel_df.loc[city]["Lat"]
    lng = indexed_hotel_df.loc[city]["Lng"]
    

        
    coordinates = f"{lat},{lng}"


    target_coordinates = coordinates
    target_search = "hotel"
    target_radius = 8000
    target_type = "lodging"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    hotel = response.json()['results'][0]['name']
    indexed_hotel_df.loc[city,'Hotel Name'] = hotel
    

In [21]:
response.json()['results'][0]['name']

'Diu Tent City - Diu, India'

In [22]:
indexed_hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,20,teknaf,1,BD,1558378758,85,20.86,92.31,84.66,8.23,Central Resort Ltd.
1,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52,Hotel National
2,118,sao filipe,2,CV,1558378776,84,14.90,-24.50,72.96,16.91,Hotel Xaguate
3,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70,Orlanova Boutique Hotel
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,"The Stag , A Unit of Deer Valley Homestay"
5,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73,OYO 236 Hotel Edmundo
6,326,cockburn town,1,TC,1558378560,79,21.46,-71.14,80.52,12.50,Osprey Beach Hotel
7,337,acapulco,5,MX,1558378854,70,16.86,-99.88,84.20,11.41,Banyan Tree Cabo Marqués
8,391,namibe,0,AO,1558378870,74,-15.19,12.15,71.70,17.40,iu Hotel Namibe
9,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50,Carrefour


In [23]:
response = response.json()
print(json.dumps(response, indent=4))

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJkCQO31kMbGP3R8SK_tYuTmnl53r2-AVd97tFYSy1n7LPuekdwiISAdWRj7OlsAPTBeW_cbeszmhN2KJpgVblzcHGcPs96kElWhWlzrj3ant5cl6K5kfH1vV2Xe0YV6MBNY0MJoORmn8on_ZWDj9zRszD-RXVF7Xt1BBQyejSqRLywvake0d8ifltbvkL9lNR7lKVNx6HwVNDpC35-qlNeBt34CqONleQDWjt2WwTuKtPDZMPxVkj0BLBIYYlqfIxAwhjgi4065t_1U1MP87AG9BUL9VL61i3gU-rNLEaOSPtNlLOBjcjJj79MbiWCWyAwxXbrcHNtpgxRf8H4iQdXHUl_Fk1MhRzne_sEruJqOB4wjBdAqHGm2hFNcG5Q69lVY9uZJheGKZcEb7QLysXgKCoWyAPGU0MI8ESIhT09EwYaMXeQlScIP6VHwU9yVg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.7021811,
                    "lng": 70.9114907
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.70387207989272,
                        "lng": 70.91277627989273
                    },
                    "southwest": {
                        "lat": 20.7011724

In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [27]:
# Add marker layer ontop of heat map
coordinates = (locations)

# Display figure
print (coordinates)

       Lat     Lng
0    68.05   39.51
1   -20.41   57.70
2    77.48  -69.36
3    40.71  112.04
4   -33.93   18.42
..     ...     ...
543  33.97   36.66
544  24.01 -104.61
545  49.65    0.71
546  51.07  132.56
547  27.72  109.18

[548 rows x 2 columns]


In [ ]:
figure_layout = {
    'width': '900px',
    'hieght': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2, center=(15,25))

In [ ]:
# Assign markersalyer to variable
markers = gmaps.marker_layer(coordinates)
    # Add the layer to the map


fig.add_layer(markers)
fig

In [25]:
# Plot Heatmap
figure_layout = {
    'width': '900px',
    'hieght': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2, center=(15,25))
# Create the heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=1.5)

# Add Layer
fig.add_layer(heat_layer)

# Display figure
plt.savefig('../images.hotel_marker.png')
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>